In [ ]:
import os
import cv2
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import zipfile
import tkinter as tk
from PIL import Image, ImageTk

data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
with zipfile.ZipFile('yolov3.weights.zip', 'r') as zip_ref:
    zip_ref.extractall(data_dir)
net = cv2.dnn.readNet(os.path.join(data_dir, "yolov3.weights"), "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers_indices = net.getUnconnectedOutLayers() 
output_layers = [layer_names[i - 1] for i in output_layers_indices]  

def perform_object_detection(frame):
    img = frame
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    recognized_labels = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            recognized_labels.append(label)
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 3)

    return img
def update_video_stream():
    ret, frame = cap.read()
    if ret:
        detected_frame = perform_object_detection(frame)
        img_rgb = cv2.cvtColor(detected_frame, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_tk = ImageTk.PhotoImage(image=img_pil)
        label_video_stream.config(image=img_tk)
        label_video_stream.image = img_tk
    label_video_stream.after(10, update_video_stream)

cap = cv2.VideoCapture(0)
root = tk.Tk()
root.title("Object Detection")
label_video_stream = tk.Label(root)
label_video_stream.pack()
update_video_stream()
close_button = tk.Button(root, text="Close", command=root.quit)
close_button.pack()
root.mainloop()
cap.release()
cv2.destroyAllWindows()
